# Model with residual connections
Training with optimized hyperparameters and prediction on the test set
---



In [8]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import pandas as pd

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Loading train, validation and test sets

In [10]:
ds = tfds.load('malaria', 
               split=('train[:80%]', 'train[80%:]'), 
               shuffle_files=False,
              data_dir='/content/gdrive/MyDrive/datasets/Malaria',
               batch_size=32,
              download=True,
              as_supervised=True,
              with_info=False)

# Model building

In [11]:
def build_model(init_filters, kernel_size, num_resid):
    '''I keep the number of filters the same in all layers in this version'''
    inputs = keras.Input(shape=(None, None, 3))
    x = keras.layers.Rescaling(scale=1.0 / 255)(inputs)
    x = keras.layers.Conv2D(
      filters=init_filters,
      kernel_size=kernel_size,
      strides=(1, 1),
      padding="same",
      activation='relu')(x)
    x = keras.layers.MaxPooling2D(
        pool_size=(2, 2), 
        strides=None, 
        padding="same")(x)
    prev_block_output = x
    for i in range(1, num_resid + 1):
        # i is the number of blocks with residual connections
      x = keras.layers.Conv2D(
          filters=init_filters,
          kernel_size=kernel_size,
          strides=(1, 1),
          padding="same",
          activation='relu')(x)
      x = keras.layers.Conv2D(
          filters=init_filters,
          kernel_size=kernel_size,
          strides=(1, 1),
          padding="same",
          activation='relu')(x)
      x = keras.layers.add([x, prev_block_output])
      prev_block_output = x
    x = keras.layers.Conv2D(
        filters=init_filters,
        kernel_size=kernel_size,
        strides=(1, 1),
        padding="same",
        activation='relu')(prev_block_output)
    x = keras.layers.GlobalMaxPooling2D()(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(units=init_filters,
                            activation="relu",
                          kernel_regularizer=None)(x)
    x = keras.layers.Dropout(0.2)(x)
    outputs = keras.layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='Adam',
         loss='binary_crossentropy',
         metrics=['accuracy'])    
    return model

In [12]:
model = build_model(64, 4, 6)

In [13]:
history = model.fit(
    ds[0],
    epochs=20,
    verbose="auto",
)

Epoch 1/20
689/689 [==============================] - 265s 383ms/step - loss: 0.3669 - accuracy: 0.7981
Epoch 2/20
689/689 [==============================] - 262s 380ms/step - loss: 0.1430 - accuracy: 0.9559
Epoch 3/20
689/689 [==============================] - 261s 379ms/step - loss: 0.1390 - accuracy: 0.9560
Epoch 4/20
689/689 [==============================] - 261s 378ms/step - loss: 0.1329 - accuracy: 0.9578
Epoch 5/20
689/689 [==============================] - 260s 377ms/step - loss: 0.1302 - accuracy: 0.9587
Epoch 6/20
689/689 [==============================] - 260s 377ms/step - loss: 0.1281 - accuracy: 0.9599
Epoch 7/20
689/689 [==============================] - 259s 376ms/step - loss: 0.1234 - accuracy: 0.9612
Epoch 8/20
689/689 [==============================] - 260s 377ms/step - loss: 0.1223 - accuracy: 0.9616
Epoch 9/20
689/689 [==============================] - 259s 376ms/step - loss: 0.1172 - accuracy: 0.9620
Epoch 10/20
689/689 [==============================] - 259s 376m

In [14]:
model.evaluate(ds[1])

173/173 [==============================] - 32s 177ms/step - loss: 0.1436 - accuracy: 0.9566


[0.14357568323612213, 0.9566400647163391]

#Conclusion
Modest improvement on the test set (0.2%)